# EXTRACCIÓN DE PARTES METAR

## En este Notebook se va a desarrollar el proceso de estracción de todos los partes METAR del Aeropuerto Adolfo Suárez Madrid-Barajas en el periodo comprendido entre el 01-11-2022 y el 31-10-2023, es decir, 1 año completo.

Un parte o código METAR es el estándar internacional en aviación del formato del código utilizado para emitir periódicamente informes de las observaciones meteorológicas en los aeródromos y aeropuertos.

Se trata de un reporte breve en forma de código alfanumérico que aporta información meteorológica detallada de un momento determinado en un aeropuerto concreto. Básicamente, es una sucesión de letras y números que se emiten periódicamente por los aeropuertos y aeródromos. 

Estos partes se emiten periódicamente, generalmente cada 30 minutos, salvo circunstancias excepcionales en las que se pueden emitir partes adicionales. 

Aunque pueden ser dificíl de descifrar para el usuario general, contienen información de gran relevancia para la actividad aérea relativa a la temperatura, precipitaciones, visibilidad en pista, viento y ráfagas entre otro. Por suerte, se pueden encontrar webs que traducen esta información a un lenguaje más amable y que además mantienen registros de los partes en el tiempo.

## Para ello se va a utilizar la técnica de extracción de datos "webscrapping" sobre la web [tutiempo](https://www.tutiempo.net/registros/lemd) que mantiene registros de estos partes desde hace vários años.


### Las librerías que se van a emplear principalmente para el proceso son:
- **selenium** 
- **joblib**
- **pandas** 


In [2]:
%pip install joblib

Note: you may need to restart the kernel to use updated packages.


In [16]:
from joblib import Parallel, delayed
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select   # seleccion de un dropdown
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
import time
import pandas as pd
import multiprocessing as mp
import warnings
warnings.filterwarnings('ignore')

from joblib._parallel_backends import LokyBackend
import asyncio

from tqdm.notebook import tqdm


## Se habilitan algunas opciones de interés para el driver

In [5]:
#driver configuration
opciones=Options()

opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)
opciones.headless=False    # si True, no aperece la ventana (headless=no visible)
opciones.add_argument('--start-maximized')         # comienza maximizado
opciones.add_argument('--incognito')

# Despues de hacer un estudio sobre la estructura de la web para plantear el proceso de extracción, se observa que en cada página se encuentran exclusivamente los partes de un día en concreto( de media unos 48).

## Además, se comprueba que al pasar de un día a otro el *link* de la web varía siguiendo un patrón claro como el siguiente:

### - ...tutiempo.net/records/lemd/{<span style="color:red">dia</span>}-{<span style="color:red">mes</span>}-{<span style="color:red">año</span>}.html

### Este formate de *links* es de especial ayuda, ya que nos va a permitir paralelizar el proceso con **joblib**, reduciendo considerablemente el tiempo de extracción.

## Vamos, por tanto, a generar los diferentes *links* necesarios, en este caso únicamente los del año 2023, de Enero a Octubre.

## De momento no nos vamos a preocupar de los días de cada mes, generando 31 links para cada uno de los meses

In [17]:
meses = ["january","february","march","april","may","june","july","august","september","october"]
dias = []
for i in range(1,32):
    dias.append(str(i))


In [18]:
urls = []
for mes in meses:
    for dia in dias:
        urls.append(f'https://en.tutiempo.net/records/lemd/{dia}-{mes}-2023.html')

In [102]:
urls[279]

'https://en.tutiempo.net/records/lemd/1-october-2023.html'

# La siguiente función <span style="color:red">extraer</span>, engloba el proceso de extraccíon de los datos necesarios para cada una de las *urls*.

## Los comentaríos de la función explican paso por paso la lógica del proceso.
## A modo de resumen, el driver entra en la página, *clicka* aceptar en los popups de cookies, extrae los datos necesarios almacenadolos en una tabla y finálmente devuelve un DataFrame con los registro METAR de ese día.

In [75]:
def extraer(url):
    table = []  # Inicializa table como una lista vacía
    columns = [] # Inicializa columns como una lista vacía
    
    try:
        # inicia el driver en la url indicada
        driver = webdriver.Chrome()
        driver.get(url)

        time.sleep(2) #espera a cargar la página
        # acepta normas
        aceptar = driver.find_element(By.XPATH,'/html/body/div[18]/div[2]/div[1]/div[2]/div[2]/button[1]')
        aceptar.click()

        time.sleep(2) #espera a cargar la página
        #acepta cookies
        aceptar = driver.find_element(By.XPATH, '//*[@id="DivAceptarCookies"]/div/a[2]')
        aceptar.click()
        
        time.sleep(2) #espera a cargar la página
        day = driver.find_elements(By.XPATH, '//table//tbody//tr')[1].text #almacena el día
        table = [row.text.split('\n')[0:3] + row.text.replace(' km/h', '').split('\n')[-1].split(' ', 2)
                 for row in driver.find_elements(By.XPATH, '//table//tbody//tr')[3::2]]  #copia los registros

        columns = ["Day", "Hour", "Condition", "Temperature", "Wind", "Relative_hum", "Pressure"] #añade las columnas

        for i in table:
            i.insert(0, day) #inserta los registros en la tabla
            
        return pd.DataFrame(table, columns=columns) #devuelve un dataframe con los datos de ese día 
        
    except Exception as e:  # Captura cualquier excepción y muestra el mensaje de error
        print(f"Error: {e}")
        print(f"Error en la URL: {url}")
        # En caso de error, simplemente se devolverá la lista vacía 'table'
        return pd.DataFrame(table, columns=columns) 
    driver.quit()
#     return pd.DataFrame(table, columns=columns)

## El siguiente código inicia el proceso de extracción usando la paralelización con 8 núcleos, es decir, realiza el proceso en *8 urls* simultáneamente.

Aunque se podría haber utilizado el proceso una única vez, se decide hacer una extracción por cada uno de los meses, mediante los índices de la tabla de urls, con el objetivo de tener un mayor control ante posibles incidencias del proceso, y poder obtener diferentes registros mensuales.

In [107]:
%%time
paralelo = Parallel(n_jobs=8, verbose=True)

lst_df = paralelo(delayed(extraer)(url) for url in tqdm(urls[279:]))

  0%|          | 0/31 [00:00<?, ?it/s]

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.


CPU times: total: 62.5 ms
Wall time: 1min 23s


[Parallel(n_jobs=8)]: Done  31 out of  31 | elapsed:  1.4min finished


## Obtenidos los DataFrame para cada unos de los días del mes, se contatenan los DataFrame y se hace una breve exploración sobre el resultado para observar si se han obtenido todos los registros necesarios.

In [108]:

pd.set_option('display.max_rows', None)
metar_october_2023 = pd.concat(lst_df)
print(len(metar_october_2023))

1513


## Finalmente se exportan a un archivo *.csv* cada uno de los DataFrames mensuales

In [110]:
metar_october_2023.to_csv("../data/metar/metar_october_2023.csv", index=False)

# A partir de aquí se repite el proceso para los meses de Noviembre y Diciembre de 2022

In [117]:
meses2 = ["november","december"]
dias2 = []
for i in range(1,32):
    dias2.append(str(i))


In [118]:
urls22 = []
for mes in meses2:
    for dia in dias2:
        urls22.append(f'https://en.tutiempo.net/records/lemd/{dia}-{mes}-2022.html')

In [128]:
urls22[31]

'https://en.tutiempo.net/records/lemd/1-december-2022.html'

In [129]:
%%time
paralelo = Parallel(n_jobs=8, verbose=True)

lst_df = paralelo(delayed(extraer)(url) for url in tqdm(urls22[31:]))

  0%|          | 0/31 [00:00<?, ?it/s]

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.


CPU times: total: 78.1 ms
Wall time: 1min 41s


[Parallel(n_jobs=8)]: Done  31 out of  31 | elapsed:  1.7min finished


In [130]:
pd.set_option('display.max_rows', None)
metar_december_2022 = pd.concat(lst_df)
# df[df.Condition == "Sunday"]
print(len(metar_december_2022))

1491


In [132]:
metar_december_2022.to_csv("../data/metar/metar_december_2022.csv", index=False)